In [12]:
# Download and extract the road data, replace the url with your data

import zipfile
import requests

def download_roads(url, road_path):
    response = requests.get(url)
    file_path = road_path + '\\' + 'roads.zip'
    with open(file_path, 'wb') as file:
        file.write(response.content)

    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(road_path)
        
download_roads('https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/trans_road_centerlines_gac/shp_trans_road_centerlines_gac.zip', 
              r'C:\School\GIS 5571\Local\Final Project Data\Data')

In [ ]:
def download_GTFS(url, GTFS_path):
    response = requests.get(url)
    file_path = GTFS_path + '\\' + 'GTFS.zip'
    with open(file_path, 'wb') as file:
        file.write(response.content)

    with zipfile.ZipFile(file_path, 'r') as zip_ref:
        zip_ref.extractall(GTFS_path)
        
download_GTFS('https://svc.metrotransit.org/mtgtfs/archive/gtfs20230923.zip', r'C:\School\GIS 5571\Local\Final Project Data\Data\GTFS')

In [10]:
# Create file geodatabse and feature dataset

def create_gdb_and_fds(path, name, spatial_ref):
    arcpy.management.CreateFileGDB(path, name)
    
    feature_dataset_name = 'NetworkFeatureDataset'
    spatial_reference = arcpy.SpatialReference(spatial_ref)
    arcpy.management.CreateFeatureDataset((path + '\\' + name), feature_dataset_name, spatial_reference)
    
#create_gdb_and_fds(r'C:\School\GIS 5571\Local\Final Project Data\Data\Network Dataset Data', 'Updated_Network1.gdb', 26915)

In [11]:
# Adding the GTFS and Street data to the new feature dataset

def add_layers_to_gdb(GTFS_Folder, road_shp):
    target_feature_dataset = 'NetworkFeatureDataset'
    arcpy.GTFSToPublicTransitDataModel_transit(GTFS_Folder, target_feature_dataset)
    
    out_feature_class = 'NetworkFeatureDataset\Streets'
    arcpy.CopyFeatures_management(road_shp, out_feature_class)
    
#add_layers_to_gdb(r'C:\School\GEOG 8294\Final Project Data Testing\Data\GTFS_Updated_Scenario_1', r'C:\School\GIS 5571\Local\Final Project Data\Data\RoadCenterline.shp')

In [12]:
# Adding the ROAD_CLASS and RestrictPedestrians fields to the streets data

def restrict_peds(input_field, input_query):
    input_dataset = 'Streets'
    arcpy.AddField_management(input_dataset, 'ROAD_CLASS', 'TEXT')
    arcpy.CalculateField_management(input_dataset, 'ROAD_CLASS', f'!{input_field}!', "PYTHON3", '')
    arcpy.AddField_management(input_dataset, 'RestrictPedestrians', "TEXT")
    
    arcpy.SelectLayerByAttribute_management(input_dataset, "NEW_SELECTION", input_query)
    
    expression = "'Y'"
    arcpy.CalculateField_management(input_dataset, 'RestrictPedestrians', expression, "PYTHON3", '')
    arcpy.SelectLayerByAttribute_management(input_dataset, "CLEAR_SELECTION")
    
#restrict_peds('CARTOCLASS', "ROAD_CLASS = 'Freeway' Or ROAD_CLASS = 'Private' Or ROAD_CLASS = 'Ramp'")

In [13]:
# Connecting the streets to the network stops and line elements

def connect_streets():
    target_feature_dataset = 'NetworkFeatureDataset'
    in_streets_features="Streets"
    search_distance="100 Meters"
    expression="RestrictPedestrians IS NULL"
    arcpy.ConnectPublicTransitDataModelToStreets_transit(target_feature_dataset, in_streets_features, search_distance, expression)
    
#connect_streets()

In [14]:
def create_ND(template_path):
    output_feature_dataset = 'NetworkFeatureDataset'
    arcpy.CreateNetworkDatasetFromTemplate_na(template_path, output_feature_dataset)
    
#create_ND(r'C:\Users\smbra\Downloads\TransitNetworkTemplate.xml')

In [15]:
def build_ND():
    in_network_dataset = 'NetworkFeatureDataset\TransitNetwork_ND'
    arcpy.BuildNetwork_na(in_network_dataset)
    
#build_ND()

In [27]:
# create the service area analysis for every input stop in the dataset

def service_area_analysis(in_stops, date, time_cutoffs, scenario_number):
    output_class = "In_Stops_Dissolve"
    arcpy.management.Dissolve(in_stops, output_class, "stop_id")
    index = 1
    
    with arcpy.da.SearchCursor(in_stops, 'arrival_time') as cursor:
        for row in cursor:
            arr_time = row[0]
    
            time_date = f'{date} {arr_time}'
            arrival_time = datetime.strptime(time_date, '%b %d %Y %H:%M:%S')

            network_data_source = 'NetworkFeatureDataset\TransitNetwork_ND'
            arcpy.MakeServiceAreaAnalysisLayer_na(network_data_source, layer_name = f'Service Area Stop Scenario {scenario_number}b {index}', cutoffs = time_cutoffs, time_of_day = arrival_time)

            in_network_analysis_layer = f'Service Area Stop Scenario {scenario_number}b {index}'
            sub_layer = 'Facilities'
            
            arcpy.AddLocations_na(in_network_analysis_layer, sub_layer, output_class)
    
            arcpy.Solve_na(in_network_analysis_layer)
            index = index + 1
            
    arcpy.management.Delete(output_class)
    
service_area_analysis(r'C:\School\GEOG 8294\Final Project Data Testing\Final_Scratchspace\Final_Scratchspace.gdb\In_Stops_Sce_2', 'Nov 1 2023', [20, 30], 2)

In [22]:
# user input for the main function

from datetime import datetime

gdb_path = r'C:\School\GIS 5571\Local\Final Project Data\Data\Network Dataset Data' # path to new geodatabase
gdb_name = 'Updated_Network2.gdb' # new geodatabase name
spatial_ref = 26915 # network dataset spatial reference
GTFS_Folder = r'C:\School\GEOG 8294\Final Project Data Testing\Data\GTFS_Updated_Scenario_2' # path to GTFS Data
roads_path = r'C:\School\GIS 5571\Local\Final Project Data\Data\RoadCenterline.shp' # path to road data
input_field = 'CARTOCLASS' # attribute name for the field that determines the road class
input_query = "ROAD_CLASS = 'Freeway' Or ROAD_CLASS = 'Private' Or ROAD_CLASS = 'Ramp'" # road class values that should be marked as 'Y', replace the road types with values in the input data
template_path = r'C:\Users\smbra\Downloads\TransitNetworkTemplate.xml'
in_stops_path = r'C:\School\GEOG 8294\Final Project Data Testing\Final_Scratchspace\Final_Scratchspace.gdb\In_Stops_Sce_2'
date = 'Nov 1 2023'
cutoffs = [20, 30]
scenario = 2

arcpy.env.workspace = f'{gdb_path}\\{gdb_name}'

In [26]:
def main(gdb_path, gdb_name, spatial_ref, GTFS_Folder, roads_path, input_field, input_query, template_path, in_stops_path, date, cutoffs, scenario):
    create_gdb_and_fds(gdb_path, gdb_name, spatial_ref)
    add_layers_to_gdb(GTFS_Folder, roads_path)
    restrict_peds(input_field, input_query)
    connect_streets()
    create_ND(template_path)
    build_ND()
    service_area_analysis(in_stops_path, date, cutoffs, scenario)
    
main(gdb_path, gdb_name, spatial_ref, GTFS_Folder, roads_path, input_field, input_query, template_path, in_stops_path, date, cutoffs, scenario) 